<a href="https://colab.research.google.com/github/wesley7790/591_rent/blob/master/V2pyGspread_%E5%90%88%E4%BD%B5%E6%8E%92%E5%BA%8F%E5%A4%9A%E9%A0%AD%E6%8E%92%E5%88%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:





# -*- coding: utf-8 -*-
"""
Created on Sun Apr 18 11:11:13 2021

@author: C101035
"""
import datetime

def getGoodinfo(url,sht,sortCol,sortKind):
    import requests

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        #'Cookie':'CLIENT%5FID=20210129095132656%5F223%2E141%2E246%2E75; LOGIN=EMAIL=wesley7792%40gmail%2Ecom&USER%5FNM=wesley+shih&ACCOUNT%5FID=115246655702074889293&ACCOUNT%5FVENDOR=Google&NO%5FEXPIRE=T; IS_TOUCH_DEVICE=F; _ga=GA1.2.1214537977.1622546002; SCREEN_SIZE=WIDTH=1680&HEIGHT=1050'
        #'Cookie':'CLIENT%5FID=20210521194811023%5F211%2E72%2E124%2E185; SCREEN_SIZE=WIDTH=1280&HEIGHT=1024; IS_TOUCH_DEVICE=F; _ga=GA1.2.1955688502.1621597697; _gid=GA1.2.278525243.1621597697; __gads=ID=114a8a03a48e4cf4:T=1621597697:S=ALNI_Mak5caAgQcFM-4PlDW-0cBxipX7cA; LOGIN=EMAIL=wesley7792%40gmail%2Ecom&USER%5FNM=%E6%96%BD%E6%96%87%E9%8C%AB&ACCOUNT%5FID=104247749873372726637&ACCOUNT%5FVENDOR=Google&NO%5FEXPIRE='
        #'Cookie':'CLIENT%5FID=20210129095132656%5F223%2E141%2E246%2E75; LOGIN=EMAIL=wesley7792%40gmail%2Ecom&USER%5FNM=wesley+shih&ACCOUNT%5FID=115246655702074889293&ACCOUNT%5FVENDOR=Google&NO%5FEXPIRE=T; _ga=GA1.2.1214537977.1622546002; TW_STOCK_BROWSE_LIST=2314%7C2102%7C1732; IS_TOUCH_DEVICE=F; SCREEN_SIZE=WIDTH=1920&HEIGHT=1080'
        #'Cookie':'CLIENT%5FID=20210222002518954%5F180%2E217%2E151%2E0; LOGIN=EMAIL=wesley7792%40gmail%2Ecom&USER%5FNM=wesley+shih&ACCOUNT%5FID=115246655702074889293&ACCOUNT%5FVENDOR=Google&NO%5FEXPIRE=T; GOOD%5FINFO%5FSTOCK%5FBROWSE%5FLIST=8%7C5227%7C4529%7C1455%7C2745%7C3362%7C006201%7C9914%7C5276; FLOAT_TITLE_BAR=F; TW_STOCK_BROWSE_LIST=4903%7C6227%7C3232%7C4133%7C1776%7C2430%7C2608%7C1229%7C1516; IS_TOUCH_DEVICE=F; SCREEN_SIZE=WIDTH=1536&HEIGHT=864',
        'referer':'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=%E6%A6%82%E5%BF%B5%E8%82%A1&INDUSTRY_CAT=3D%E2%80%933D%E9%9B%BB%E8%A6%96'
        }
    #使用headers讓網站辨識我們是存在的使用者
    res = requests.get(url,headers = headers)
    #更改資料的字元編碼
    res.encoding = 'utf-8'
    import bs4

    soup = bs4.BeautifulSoup(res.text,'lxml')
    #用css的語法找到id = 'txtFinDetailData'
    data = soup.select_one('#txtStockListData')

    import pandas as pd
    #用.prettify()將data整理得更好且用read_html來找到表格
    df = pd.read_html(data.prettify())
    #看有幾個表格
    len(df)
    #第[1]表格為我們的目標。
    dfs = df[1]
    #整理表格的標頭
    dfs.columns = dfs.columns.get_level_values(0)

    i = 18
    while i < len(dfs):
      dfs=dfs.drop(dfs.index[[i]])
      i=i+18


    dfs=dfs.apply(lambda s: pd.to_numeric(s.astype(str).str.replace("↗", "").str.replace("↘", "").str.replace("+", "").str.replace("→", ""), errors='ignore'))


    #顯示頭幾筆資料
    #dfs.head()
    import gspread
    from gspread_dataframe import set_with_dataframe
    from oauth2client.service_account import ServiceAccountCredentials as sac
    auth_json = '/content/drive/My Drive/Colab Notebooks/wesley06188A.json'

    gs_scopes = ['https://spreadsheets.google.com/feeds']
    # 連線資料表
    cr = sac.from_json_keyfile_name(auth_json, gs_scopes)
    gc = gspread.authorize(cr)
    # 開啟資料表
    spreadsheet_key = '1PeLl4HME4Zpu4eU0cwAEUYAblOJt6j3G0g95FfFpJrU'
    sheet = gc.open_by_key(spreadsheet_key)
    # 開啟工作簿
    wks = sheet.worksheet(sht)
    # 清除所有內容
    cell_clear='A1:R'+str(wks.row_count)
    cell_sort='A2:V'+str(wks.row_count)
    sheet.values_clear(wks.title+"!"+ cell_clear)

    # APPEND DATA TO SHEET
    set_with_dataframe(wks, dfs) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET


    wks.sort((sortCol, sortKind),range=cell_sort)
    #第一列標題不刪
    #wks.delete_rows(1)
    print(sht)
print(datetime.datetime.now())
url="https://goodinfo.tw/tw2/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%97%A5K%E7%B7%9A%E5%90%91%E4%B8%8A%E7%AA%81%E7%A0%B4%E5%AD%A3%E7%B7%9A%40%40%E6%97%A5K%E7%B7%9A%E5%90%91%E4%B8%8A%E7%AA%81%E7%A0%B4%E5%9D%87%E5%83%B9%E7%B7%9A%40%40%E5%AD%A3%E7%B7%9A"
getGoodinfo(url,'日線突破季線',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%97%A5K%E7%B7%9A%E8%B7%B3%E7%A9%BA%E4%B8%8A%E6%BC%B2%40%40%E8%B7%B3%E7%A9%BA%E4%B8%8A%E6%BC%B2%E2%80%93%E6%97%A5K%E7%B7%9A%40%40%E8%B7%B3%E7%A9%BA%E4%B8%8A%E6%BC%B2"
getGoodinfo(url,'日K線跳空上漲',1,'des')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=5%E6%97%A5%2F10%E6%97%A5%2F%E6%9C%88%E7%B7%9A%E5%A4%9A%E9%A0%AD%E6%8E%92%E5%88%97%40%40%E5%9D%87%E5%83%B9%E7%B7%9A%E5%A4%9A%E9%A0%AD%E6%8E%92%E5%88%97%40%405%E6%97%A5%2F10%E6%97%A5%2F%E6%9C%88"
getGoodinfo(url,'五十月均多頭排列',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E4%B8%89%E5%A4%A7%E6%B3%95%E4%BA%BA%E9%80%A3%E8%B2%B7+%E2%80%93+%E6%97%A5%40%40%E4%B8%89%E5%A4%A7%E6%B3%95%E4%BA%BA%E9%80%A3%E7%BA%8C%E8%B2%B7%E8%B6%85%40%40%E4%B8%89%E5%A4%A7%E6%B3%95%E4%BA%BA%E9%80%A3%E7%BA%8C%E8%B2%B7%E8%B6%85+%E2%80%93+%E6%97%A5"
getGoodinfo(url,'法人連買_日',19,'asc')

#url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%97%A5%E6%88%90%E4%BA%A4%E5%BC%B5%E6%95%B8%E5%89%B5%E8%BF%91%E6%9C%9F%E6%96%B0%E9%AB%98%E6%97%A5%E6%95%B8%40%40%E6%88%90%E4%BA%A4%E5%BC%B5%E6%95%B8%40%40%E6%97%A5%E6%88%90%E4%BA%A4%E5%BC%B5%E6%95%B8%E5%89%B5%E8%BF%91%E6%9C%9F%E6%96%B0%E9%AB%98%E6%97%A5%E6%95%B8"
#getGoodinfo(url,'成交張數創近期新高日數',2,'asc')

#url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E9%87%8F%E5%A2%9E%E5%8A%A0%E5%BC%B5%E6%95%B8%E2%80%93%E7%95%B6%E6%97%A5%E6%88%90%E4%BA%A4%E9%87%8F%E8%88%87%E6%98%A8%E6%97%A5%E6%AF%94%40%40%E6%88%90%E4%BA%A4%E9%87%8F%E5%A2%9E%E5%8A%A0%E5%BC%B5%E6%95%B8%40%40%E7%95%B6%E6%97%A5%E6%88%90%E4%BA%A4%E9%87%8F%E8%88%87%E6%98%A8%E6%97%A5%E6%AF%94"
#getGoodinfo(url,'當日成交量與昨日比',2,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E9%95%B7%E7%B4%85%28%E6%A3%92%E5%B9%85%3E3%2E5%25%29%E2%80%93%E7%95%B6%E6%97%A5%40%40%E5%96%AE%E4%B8%80K%E7%B7%9A%E5%9E%8B%E6%85%8B%E2%80%93%E6%97%A5%E7%B7%9A%40%40%E7%95%B6%E6%97%A5%EF%BC%9A%E9%95%B7%E7%B4%85%28%E6%A3%92%E5%B9%85%3E3%2E5%25%29"
getGoodinfo(url,'長紅棒_當日',16,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E9%95%B7%E4%B8%8B%E5%BD%B1%E7%B7%9A%28%E7%B7%9A%E5%B9%85%3E3%2E5%25%29%E2%80%93%E7%95%B6%E6%97%A5%40%40%E5%96%AE%E4%B8%80K%E7%B7%9A%E5%9E%8B%E6%85%8B%E2%80%93%E6%97%A5%E7%B7%9A%40%40%E7%95%B6%E6%97%A5%EF%BC%9A%E9%95%B7%E4%B8%8B%E5%BD%B1%E7%B7%9A%28%E7%B7%9A%E5%B9%85%3E3%2E5%25%29"
getGoodinfo(url,'長下影線',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A85%E6%97%A5%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%40%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A8%E5%9D%87%E5%83%B9%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%405%E6%97%A5%E7%B7%9A"
getGoodinfo(url,'5日線之上',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A810%E6%97%A5%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%40%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A8%E5%9D%87%E5%83%B9%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%4010%E6%97%A5%E7%B7%9A"
getGoodinfo(url,'10日線之上',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A8%E6%9C%88%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%40%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A8%E5%9D%87%E5%83%B9%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%40%E6%9C%88%E7%B7%9A"
getGoodinfo(url,'月線之上',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A8%E5%AD%A3%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%40%E6%88%90%E4%BA%A4%E5%83%B9%E5%9C%A8%E5%9D%87%E5%83%B9%E7%B7%9A%E4%B9%8B%E4%B8%8A%40%40%E5%AD%A3%E7%B7%9A"
getGoodinfo(url,'季線之上',1,'asc')



url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%97%A5KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89%40%40%E6%97%A5KD%E7%9B%B8%E4%BA%92%E4%BA%A4%E5%8F%89%40%40KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89"
getGoodinfo(url,'日KD交叉',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%9C%88KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89%40%40%E6%9C%88KD%E7%9B%B8%E4%BA%92%E4%BA%A4%E5%8F%89%40%40KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89"
getGoodinfo(url,'月KD交叉',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E5%AD%A3KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89%40%40%E5%AD%A3KD%E7%9B%B8%E4%BA%92%E4%BA%A4%E5%8F%89%40%40KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89"
getGoodinfo(url,'季KD交叉',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E5%B9%B4KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89%40%40%E5%B9%B4KD%E7%9B%B8%E4%BA%92%E4%BA%A4%E5%8F%89%40%40KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89"
getGoodinfo(url,'年KD交叉',1,'asc')




url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=OSC%E7%94%B1%E8%B2%A0%E8%BD%89%E6%AD%A3%40%40%E6%97%A5MACD%E8%90%BD%E9%BB%9E%40%40OSC%E7%94%B1%E8%B2%A0%E8%BD%89%E6%AD%A3"
getGoodinfo(url,'日OSC由負轉正',1,'asc')

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=OSC%E7%94%B1%E8%B2%A0%E8%BD%89%E6%AD%A3%40%40%E9%80%B1MACD%E8%90%BD%E9%BB%9E%40%40OSC%E7%94%B1%E8%B2%A0%E8%BD%89%E6%AD%A3"
getGoodinfo(url,'週OSC由負轉正',1,'asc')



url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%97%A5RSI%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89%40%40%E6%97%A5RSI%E7%9B%B8%E4%BA%92%E4%BA%A4%E5%8F%89%40%40RSI%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89"
getGoodinfo(url,'日RSI黃金交叉',7,'asc')


url="https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=&FL_VAL_S0=&FL_VAL_E0=&FL_ITEM1=&FL_VAL_S1=&FL_VAL_E1=&FL_ITEM2=&FL_VAL_S2=&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81%7C%7C%E6%88%90%E4%BA%A4%E9%87%8F%E5%A2%9E%E5%8A%A0%E5%B9%85%E5%BA%A6%E2%80%93%E7%95%B6%E6%97%A5%E6%88%90%E4%BA%A4%E9%87%8F%E8%88%87%E6%98%A8%E6%97%A5%E6%AF%94%40%40%E6%88%90%E4%BA%A4%E9%87%8F%E5%A2%9E%E5%8A%A0%E5%B9%85%E5%BA%A6%40%40%E7%95%B6%E6%97%A5%E6%88%90%E4%BA%A4%E9%87%8F%E8%88%87%E6%98%A8%E6%97%A5%E6%AF%94%7C%7C1000&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E8%BF%9112%E6%97%A5%E6%88%90%E4%BA%A4%E9%87%8F%E4%B8%80%E8%A6%BD&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
getGoodinfo(url,'成交量增幅',1,'asc')
"""1130223新版週KD已有張數，故張數600以上免查。
url="https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=%E8%87%AA%E8%A8%82%E7%AF%A9%E9%81%B8&INDUSTRY_CAT=%E6%88%91%E7%9A%84%E6%A2%9D%E4%BB%B6&FL_ITEM0=%E6%88%90%E4%BA%A4%E5%BC%B5%E6%95%B8+%28%E5%BC%B5%29&FL_VAL_S0=0&FL_VAL_E0=1000&FL_VAL_CHK0=T&FL_ITEM1=&FL_VAL_S1=&FL_VAL_E1=&FL_ITEM2=&FL_VAL_S2=&FL_VAL_E2=&FL_ITEM3=&FL_VAL_S3=&FL_VAL_E3=&FL_ITEM4=&FL_VAL_S4=&FL_VAL_E4=&FL_ITEM5=&FL_VAL_S5=&FL_VAL_E5=&FL_ITEM6=&FL_VAL_S6=&FL_VAL_E6=&FL_ITEM7=&FL_VAL_S7=&FL_VAL_E7=&FL_ITEM8=&FL_VAL_S8=&FL_VAL_E8=&FL_ITEM9=&FL_VAL_S9=&FL_VAL_E9=&FL_ITEM10=&FL_VAL_S10=&FL_VAL_E10=&FL_ITEM11=&FL_VAL_S11=&FL_VAL_E11=&FL_RULE0=&FL_RULE1=&FL_RULE2=&FL_RULE3=&FL_RULE4=&FL_RULE5=&FL_RANK0=&FL_RANK1=&FL_RANK2=&FL_RANK3=&FL_RANK4=&FL_RANK5=&FL_FD0=&FL_FD1=&FL_FD2=&FL_FD3=&FL_FD4=&FL_FD5=&FL_SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&FL_SHEET2=%E6%97%A5&FL_MARKET=%E4%B8%8A%E5%B8%82%2F%E4%B8%8A%E6%AB%83&FL_QRY=%E6%9F%A5++%E8%A9%A2"
getGoodinfo(url,'張數600以上',1,'asc')
"""

url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E9%80%B1KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89%40%40%E9%80%B1KD%E7%9B%B8%E4%BA%92%E4%BA%A4%E5%8F%89%40%40KD%E9%BB%83%E9%87%91%E4%BA%A4%E5%8F%89"
getGoodinfo(url,'週KD交叉',12,'asc')



2024-04-04 00:13:55.404773
日線突破季線
日K線跳空上漲
五十月均多頭排列
法人連買_日
長紅棒_當日
長下影線
5日線之上
10日線之上
月線之上
季線之上
日KD交叉


In [ ]:
pip install --upgrade gspread==4.0.0

  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
"""
#url="https://goodinfo.tw/StockInfo/StockList.asp?RPT_TIME=&MARKET_CAT=%E6%99%BA%E6%85%A7%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E6%97%A5K%E7%B7%9A%E8%B7%B3%E7%A9%BA%E4%B8%8B%E8%B7%8C%40%40%E8%B7%B3%E7%A9%BA%E4%B8%8B%E8%B7%8C%E2%80%93%E6%97%A5K%E7%B7%9A%40%40%E8%B7%B3%E7%A9%BA%E4%B8%8B%E8%B7%8C"
#getGoodinfo(url,'日K線跳空下跌',1,'asc')
"""

#url="https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=%E6%88%91%E7%9A%84%E9%81%B8%E8%82%A1&INDUSTRY_CAT=%E7%B6%B2%E7%AB%99%E4%B9%9F%E8%83%BD%E6%8C%96%E5%87%BA%E6%9C%88%E6%BC%B2%E5%B9%8560%25%E7%9A%84%E9%A3%86%E8%82%A1&SHEET=%E5%AD%A3%E7%8D%B2%E5%88%A9%E8%83%BD%E5%8A%9B&SHEET2=%E7%8D%B2%E5%88%A9%E8%83%BD%E5%8A%9B%20(%E5%AD%A3%E5%A2%9E%E6%B8%9B%E7%B5%B1%E8%A8%88)&RPT_TIME=%E6%9C%80%E6%96%B0%E8%B3%87%E6%96%99"
#getGoodinfo(url,'月漲幅60%的飆股',1,'asc')